# www http://brainbashers.com/skyscrapers.asp
get e.g. a puzzle from ^^^
and check there your solution: ==> green square if ok
#example end of this file

# notebook with demonstration of 4 puzzles ;-)
by PKHG

In [ ]:
from SudokuBaseClass import * #see result of this cell ;-)

# self. because the result of maakStartVals is later used in th solver class
def maakStartVals(s):
    """
        make empty startvalue example for ScyscrInfo
    """
    w = ['self.U', 'self.R', 'self.D', 'self.L']
    result = []
    for LL in w:
        tmp = LL + ';'
        for i in range(s - 1):
            tmp += '0 '
        tmp += '0'
        result.append(tmp)
    result.append('self.instance;x,y,w 0,0,0') 
    print('zero based counting! first column is col number 0 etc.')
    print('x,y zero based coordinates w given border value')                 
    print('if 2 or more bordervalues are given use ONE space as seperator token')
    print('have a look of the possible examples, in this notebook')
    return result

SC = SudokuConditions()
print('from initializing SC SC.size {}'.format(SC.size))

In [ ]:
#ScyscrInfo = []

def example(size):
    """
    4 examplels of size 3, 4, 8, 9 possible
    usage: size, ScyscrInfo = example(x)
    x in {3, 4, 8, 9}
    """
    if size == 8:
        ScyscrInfo = ['self.U;3 3 0 0 2 3 2 4',
            'self.R;0 0 0 3 2 0 0 3',
            'self.D;5 0 0 0 0 0 3 2',
            'self.L;3 3 0 0 0 0 2 0',
            'self.instance;0,7,4 1,6,1 2,0,6 3,4,1 3,5,7 4,2,2 5,3,6 6,6,5'
            ]

    elif size == 3:
        ScyscrInfo = ['self.U;3 2 1',
          'self.R;0 2 2',
          'self.D;0 0 0',
          'self.L;0 0 0',
          'self.instance;'
         ]
    elif size == 4:
        ScyscrInfo = ['self.U;3 0 3 0', 'self.R;0 1 0 0', 'self.D;0 0 0 0',
                      'self.L;0 0 3 0', 'self.instance;']
    elif size == 9:
        ScyscrInfo = ['self.U;1 2 3 3 4 3 2 2 4',
                  'self.R;4 3 2 2 3 3 4 3 1', 
                  'self.D;4 4 2 3 2 3 3 3 1', 
                  'self.L;1 2 4 2 3 3 2 4 5', 
                  'self.instance;0,1,3 0,8,4 1,2,1 1,4,6 1,8,7 2,0,2 2,1,6 3,2,5 3,6,4 3,7,2 3,8,8 4,0,6 4,2,2 4,3,5 5,6,9 5,7,5 6,7,3 7,1,5 7,5,8 8,0,3 8,3,2 8,7,6']
    return (size, ScyscrInfo)

size, ScyscrInfo = example(size = 3)
size, ScyscrInfo

In [3]:
class z3_PKHG_Scy_solver(SudokuConditions):
    def __init__(self, size, DBG, Info):
        SudokuConditions.__init__(self, size, DBG)
        self.ScyscrInfo = Info
        #DBG print(self.size)
        #DBG print(self.instance)
        self.U, self.R, self.D, self.L = self.makeRanden() #yet empty
        self.fillRandenInstance()
        self.instance_c = [ If(self.instance[i][j] == 0,
                  True,
                  self.A[i][j] == self.instance[i][j])
               for i in range(size) for j in range(self.size) ]
        self.elmTODO = self.givenElements()
        #print('DBG PKHG LL14 elmTODO {}'.format(self.elmTODO))
        self.scyscraper_c = self.makeCondData()
        self.stDD = self.cells_c + self.rows_c + self.cols_c + self.instance_c
        
        print('\n\nSolving prepared')
     
    def makeCondData(self):
        size = self.size
        end_result = []
        def w_elements_str(side, nr): #rembemer, this is for 6x6 puzzle
            result = []
            if side == 'self.U': #from above to below
                for i in range(size):
                    #res = ('AA[' + str(i) + '][' + str(nr) + ']')
                    result.append('self.A[' + str(i) + '][' + str(nr) + ']') # U measn from 1 to size downwards colum nr
            elif side == 'self.L':  # from left to right
                for i in range(size):
                    #result.append(A[nr][i])
                    result.append('self.A[' + str(nr) + '][' + str(i) + ']')
            elif side == 'self.R':  #form right to left
                for i in range(size -1, -1, -1):
                    #result.append(A[nr][i]) # U measn from 1 to size downwards colum nr
                    result.append('self.A[' + str(nr) + '][' + str(i) + ']')
            elif side == 'self.D':  #form right to left
                for i in range(size -1, -1, -1):
                    result.append('self.A[' + str(i) + '][' + str(nr) + ']')
            return result

        doThese = self.elmTODO
        #DBG print('DBGLL43 doThese {} + len {}'.format(doThese, len(doThese)))
        #DBGLL43 doThese [['self.U', [0, 3], [1, 2], [2, 1]], ['self.L'],
        #['self.R', [1, 2], [2, 2]], ['self.D']]
        for toth in doThese:
            if len(toth) == 1:
                #DBG print('DBG LL49 eval(toth[0]) {}'.format(eval(toth[0])))
                #DBG LL46 eval(toth[0]) [d0, d1, d2]  #OK
                continue
            #DBG print('DBG LL53 REST toth {}'.format(toth[1:]))
            name = toth[0]
            #DBG print('DBFG LL76 name = {}'.format(name))
            for doGeg in toth[1:]:
                #DBG print('DBG LL57 doGeg = {} '.format(doGeg))
                result = []
                nr = doGeg[0]
                if name == 'self.U': #from above to below
                    for i in range(size):
                        #res = ('AA[' + str(i) + '][' + str(nr) + ']')
                        result.append('self.A[' + str(i) + '][' + str(nr) + ']') # U measn from 1 to size downwards colum nr
                elif name == 'self.L':  # from left to right
                    for i in range(size):
                        #result.append(A[nr][i])
                        result.append('self.A[' + str(nr) + '][' + str(i) + ']')
                elif name == 'self.R':  #form right to left
                    for i in range(size -1, -1, -1):
                        #result.append(A[nr][i]) # U measn from 1 to size downwards colum nr
                        result.append('self.A[' + str(nr) + '][' + str(i) + ']')
                elif name == 'self.D':  #form right to left
                    for i in range(size -1, -1, -1):
                        result.append('self.A[' + str(i) + '][' + str(nr) + ']')
                #DBG print('DBG LL95 result {}'.format(result))
                result = [result]
                result.append(doGeg[1])
                #now the ScyScraper for z3 has to be created
                newResult = [] #initialize
                bgi = result
                #DBG print('DBG LL80 bgi = {}'.format(bgi))
                
                l_As, l_val = bgi
                #DBG print('DBG 15:47 LL106 l_As = {} l_val = {}'.format(l_As, l_val))
                
                tijdelijk = [1, eval('If(' +  l_As[1] + ' > ' + l_As[0] + ', 1, 0)')]
                #tmp = result[1]
                newResult = tijdelijk
                #DBG print('DBG PKHGTIJDELIJK LL111 tijdelijk {}'.format(tijdelijk))
                      
                #DBG print('DBG LL135  tmp = {} newResult = {}'.format(tmp, newResult))
                helper = 'self.myMax(' + l_As[0] + ',' + l_As[1] + ')'
                #DBG print('DBG LL138 l_As[2:] = {}'.format(l_As[2:]))
                for i, el in enumerate(l_As[2:]):
                    newResult.append(eval('If(' + el + ' > ' + helper + ', 1, 0 )'))
                    helper = 'self.myMax(' + l_As[i + 2] + ', ' + helper + ')'
                end_result.append(Sum(newResult) == l_val)
                #DBG print('DBGLL114 end_result[-1] = {}'.format(end_result[-1]))
                
        return end_result
                #res = w_elements_str(name, doGeg)
                #print('dbg LL79 res = {}'.format(res))
  
    def myMax(self, A, B):
         return If(A > B, A, B)  
        
    def makeRanden(self):
        """
        make 4 empty z3 borderarrays ,e.g. self.U] = [u0, u1, ... u(size-1) ] 
        """
        size = self.size
        result = []
        randen = ['u','r','d','l']
        for letter in randen:
            capL = 'self.' + letter.upper()
            tmp = capL + ' = [ Int("' + letter + '%s" % (i)) for i in range('+ str(size) + ')]'
            #DBG print(tmp)
            exec(tmp)
            result.append(eval(capL))
        return result

    def fillRandenInstance(self):
        """
        z3Bordernames alreay done in __init__
        given bordernumbers are inserted
        """
        bordernames = ['self.U', 'self.R', 'self.D', 'self.L']
        for i, el in enumerate(self.ScyscrInfo):
            if i < 4:
                thisBorderInfo = el[7:].split(' ')
                for j in range(self.size):
                    tmp = bordernames[i] + '[' + str(j) + ']='
                    if thisBorderInfo[j] == '0': #not given border
                        continue
                    tmp += thisBorderInfo[j] 
                    #DBG print('DBG LL40 tmp {}'.format(tmp))
                    exec(tmp)
                #vals = el[7:].split(' ')
                #DBG print(vals)
            else:
                #DBG print('len(el) {} '.format(len(el[14:])))
                if len(el) == 14:
                    print('Info: no internal numbers given')
                else:
                    triples = el[14:].split(' ')
                    #8x8: ['0,7,4', '1,6,1', '2,0,6', '3,4,1', '3,5,7', '4,2,2', '5,3,6', '6,6,5']
                    #DBG print('DBG LL51 triples = {}'.format(triples))
                    for cova in triples:
                        xx,yy,ww = cova.split(',')
                        self.instance[int(xx)][int(yy)] = int(ww)
    
    def givenElements(self):
        elm = ['self.U', 'self.L', 'self.R', 'self.D']
        results = []
        for name in elm:
            tmp = [name]
            obj = eval(name)
            for i, el in enumerate(obj):
                if type(el) == type(1):
                    tmp.append([i, el])
            results.append(tmp)
        return results

    def runthesolver(self, countmax=2):   
        self.countmax = countmax
        startdata = self.stDD.copy()  + self.scyscraper_c
        size = self.size
        count = 0
        countmax = 2 #if there are more than on solution
        z3solver = Solver()
        z3solver.add(startdata)
        while z3solver.check() == sat and count < self.countmax:
            count += 1
            print('count = ', count)
            m = z3solver.model()
            r = [ [ m.evaluate(self.A[i][j]) for j in range(size) ]  for i in range(size) ]
            for rij in r:
                print(rij)
            vals = [self.A[i][j] == m.evaluate(self.A[i][j]) for j in range(size)  for i in range(size)]
            z3solver.add(Not(And(vals)))
        if count == 0:
            print('unsat')
        elif count == 1:
            print("\n====thanks to z3 ;-) =========\n")
            print('THE solution found')
        elif count <= self.countmax:
            print('all solutions found, wrong puzzle or input errors')
        else:
            print('count = {} '.format(count), '\nmaybe more solutions possible!')
  
                                             
        

In [4]:
size, ScyscrInfo = example(size = 9)
size, ScyscrInfo

(9,
 ['self.U;1 2 3 3 4 3 2 2 4',
  'self.R;4 3 2 2 3 3 4 3 1',
  'self.D;4 4 2 3 2 3 3 3 1',
  'self.L;1 2 4 2 3 3 2 4 5',
  'self.instance;0,1,3 0,8,4 1,2,1 1,4,6 1,8,7 2,0,2 2,1,6 3,2,5 3,6,4 3,7,2 3,8,8 4,0,6 4,2,2 4,3,5 5,6,9 5,7,5 6,7,3 7,1,5 7,5,8 8,0,3 8,3,2 8,7,6'])

In [5]:
#M = None
M = z3_PKHG_Scy_solver(size,1, ScyscrInfo)

got: size = 9
A, cells_c, rows_c, cols_c , instance NOW in SC available


Solving prepared


In [6]:
M.runthesolver()

count =  1
[9, 3, 6, 1, 2, 5, 8, 7, 4]
[5, 9, 1, 8, 6, 2, 3, 4, 7]
[2, 6, 8, 3, 5, 4, 7, 9, 1]
[7, 1, 5, 9, 3, 6, 4, 2, 8]
[6, 7, 2, 5, 4, 9, 1, 8, 3]
[1, 8, 4, 6, 7, 3, 9, 5, 2]
[8, 2, 9, 4, 1, 7, 6, 3, 5]
[4, 5, 3, 7, 9, 8, 2, 1, 6]
[3, 4, 7, 2, 8, 1, 5, 6, 9]

====thanks to z3 ;-) =========

THE solution found


In [7]:
len(M.scyscraper_c)

36

In [8]:
M.U, M.R, M.D, M.L, M.instance

([1, 2, 3, 3, 4, 3, 2, 2, 4],
 [4, 3, 2, 2, 3, 3, 4, 3, 1],
 [4, 4, 2, 3, 2, 3, 3, 3, 1],
 [1, 2, 4, 2, 3, 3, 2, 4, 5],
 [[0, 3, 0, 0, 0, 0, 0, 0, 4],
  [0, 0, 1, 0, 6, 0, 0, 0, 7],
  [2, 6, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 5, 0, 0, 0, 4, 2, 8],
  [6, 0, 2, 5, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 9, 5, 0],
  [0, 0, 0, 0, 0, 0, 0, 3, 0],
  [0, 5, 0, 0, 0, 8, 0, 0, 0],
  [3, 0, 0, 2, 0, 0, 0, 6, 0]])

In [9]:
for el in dir(M):
    if el.startswith('_'):
        continue
    print(el)

A
D
L
R
ScyscrInfo
U
cells_c
cols_c
countmax
elmTODO
fillRandenInstance
givenElements
instance
instance_c
makeCondData
makeRanden
myMax
randen
rows_c
runthesolver
scyscraper_c
size
stDD


In [10]:
for exam in [3,4,8,9]:
    size, ScyscrInfo = example(size = exam)
    size, ScyscrInfo
    M = z3_PKHG_Scy_solver(size,1, ScyscrInfo)
    M.runthesolver()

got: size = 3
A, cells_c, rows_c, cols_c , instance NOW in SC available
Info: no internal numbers given


Solving prepared
count =  1
[1, 2, 3]
[2, 3, 1]
[3, 1, 2]

====thanks to z3 ;-) =========

THE solution found
got: size = 4
A, cells_c, rows_c, cols_c , instance NOW in SC available
Info: no internal numbers given


Solving prepared
count =  1
[2, 4, 1, 3]
[3, 1, 2, 4]
[1, 3, 4, 2]
[4, 2, 3, 1]

====thanks to z3 ;-) =========

THE solution found
got: size = 8
A, cells_c, rows_c, cols_c , instance NOW in SC available


Solving prepared
count =  1
[5, 1, 7, 8, 2, 6, 3, 4]
[2, 7, 6, 4, 8, 3, 1, 5]
[6, 5, 3, 2, 4, 1, 8, 7]
[8, 6, 4, 5, 1, 7, 2, 3]
[7, 8, 2, 1, 3, 5, 4, 6]
[4, 3, 1, 6, 5, 2, 7, 8]
[3, 2, 8, 7, 6, 4, 5, 1]
[1, 4, 5, 3, 7, 8, 6, 2]

====thanks to z3 ;-) =========

THE solution found
got: size = 9
A, cells_c, rows_c, cols_c , instance NOW in SC available


Solving prepared
count =  1
[9, 3, 6, 1, 2, 5, 8, 7, 4]
[5, 9, 1, 8, 6, 2, 3, 4, 7]
[2, 6, 8, 3, 5, 4, 7, 9, 1]
[7, 1,

In [12]:
!ls sol*.*

sol4x4_31aug.png sol6x6_14aug.png sol8x8_20aug.png sol9x9_0609.png


# 4x4 test 31 src contains date
# try this one, (my first 6x6 solved) ;-)
<img src="sol6x6_14aug.png" />

<img src='sol4x4_31aug.png'/>

<img src="sol8x8_20aug.png"/>

<img src="sol9x9_0609.png">

In [13]:
maakStartVals(6)

zero based counting! first column is col number 0 etc.
x,y zero based coordinates w given border value
if 2 or more bordervalues are given use ONE space as seperator token
have a look of the possible examples, in this notebook


['self.U;0 0 0 0 0 0',
 'self.R;0 0 0 0 0 0',
 'self.D;0 0 0 0 0 0',
 'self.L;0 0 0 0 0 0',
 'self.instance;x,y,w 0,0,0']

# new special scyscraper puzzle (see picture)
<img src="start6x6_1408.png"/>

In [14]:
# From left to right resp. up to down
ScyscrInfo = [
 'self.U;0 0 0 0 3 0', #the 4 removed for more solutions (exactly 4 ;-) )
 'self.R;3 0 5 0 4 0', # if 1,3,1 is given ==> one solution!
 'self.D;0 0 0 0 3 3',
 'self.L;3 0 1 2 0 3',
 'self.instance;'] #two solutions   one condition more needed e.g. next line
 #'self.instance;1,3,1'] #Without unsat ? no inside values?!

In [15]:
M4 = z3_PKHG_Scy_solver(6 , 0, ScyscrInfo)

A, cells_c, rows_c, cols_c , instance NOW in SC available
Info: no internal numbers given


Solving prepared


In [16]:
M4.runthesolver(countmax = 5)

count =  1
[1, 5, 2, 6, 4, 3]
[3, 2, 4, 1, 5, 6]
[6, 4, 5, 3, 2, 1]
[4, 1, 3, 2, 6, 5]
[5, 6, 1, 4, 3, 2]
[2, 3, 6, 5, 1, 4]
count =  2
[2, 1, 5, 6, 4, 3]
[3, 4, 1, 2, 5, 6]
[6, 5, 4, 3, 1, 2]
[4, 2, 3, 1, 6, 5]
[5, 6, 2, 4, 3, 1]
[1, 3, 6, 5, 2, 4]
count =  3
[1, 5, 2, 6, 4, 3]
[3, 1, 4, 2, 5, 6]
[6, 4, 5, 3, 2, 1]
[4, 2, 3, 1, 6, 5]
[5, 6, 1, 4, 3, 2]
[2, 3, 6, 5, 1, 4]
count =  4
[1, 5, 2, 6, 4, 3]
[3, 2, 1, 4, 5, 6]
[6, 4, 5, 3, 2, 1]
[4, 1, 3, 2, 6, 5]
[5, 6, 4, 1, 3, 2]
[2, 3, 6, 5, 1, 4]
all solutions found, wrong puzzle or input errors


<img src="sol6x6_14aug.png" />